In [ ]:
#Import Libraries

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 180 kB 45.6 MB/s 
     |████████████████████████████████| 133 kB 46.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=40d0c61f684108da3f9df9a2c553ba471942677860da7f751dae6cb44f35d055
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=bac8752ecc59dc2ee88a1a91548c3d06f9b52071b8db86c5378c89614e7b1f41
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [ ]:
import wandb

In [ ]:
# Log in to your W&B account
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [2, 5]
        },
        'batch_size': {
            'values': [256, 128]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4,3e-5, 1e-5]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
    }
}

In [ ]:
kwargs={}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Pytorch-sweeps-example")

Create sweep with ID: r5ce7fu8
Sweep URL: https://wandb.ai/dolcelatte/Pytorch-sweeps-example/sweeps/r5ce7fu8


In [ ]:
def train():
  config_defaults={}
  config_defaults['batch_size']=256
  config_defaults['test_batch_size']=256
  config_defaults['epochs']=5  #The number of Epochs is the number of times you go through the full dataset. 
  config_defaults['lr']=0.01 #Learning rate is how fast it will decend. 
  config_defaults['optimizer']= 'adam'
  config_defaults['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).
  config_defaults['cuda']=True
  wandb.init(
      project="mnist example",
      config=config_defaults)
  config = wandb.config

  #This defines the structure of the NN.
  class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)
              
  model = Net()

    #load the data
  train_loader = torch.utils.data.DataLoader(
                    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                    batch_size=config.batch_size, shuffle=True, **kwargs)

  test_loader = torch.utils.data.DataLoader(
                    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                    batch_size=config.test_batch_size, shuffle=True, **kwargs)

  if config_defaults['cuda']:
      model.cuda()

  if config.optimizer=='sgd':
    optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=config.momentum)
  elif config.optimizer=='adam':
    optimizer = optim.Adam(model.parameters(),lr=config.learning_rate)

  for epoch in range(1, config.epochs + 1):
      model.train()
      for batch_idx, (data, target) in enumerate(train_loader):
          if config_defaults['cuda']:
              data, target = data.cuda(), target.cuda()
          data, target = Variable(data), Variable(target)
          optimizer.zero_grad()
          output = model(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          optimizer.step()
          wandb.log({"train_loss": loss})
      model.eval()
      test_loss = 0
      correct = 0
      for data, target in test_loader:
          if config_defaults['cuda']:
              data, target = data.cuda(), target.cuda()
          data, target = Variable(data, volatile=True), Variable(target)
          output = model(data)
          test_loss += F.nll_loss(output, target, size_average=False)
          pred = output.data.max(1, keepdim=True)[1] 
          correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

      test_loss /= len(test_loader.dataset)
      wandb.log({"val_loss": test_loss})

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: zoyec6h6 with config:
wandb: 	batch_size: 256
wandb: 	epochs: 2
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: Currently logged in as: dolcelatte (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



...

train_loss,███▇▇▆▅▅▅▄▃▃▃▂▃▃▂▃▃▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▂▂▁▁▁▂
val_loss,█▃▂▁▁


wandb: Agent Starting Run: a8z91irg with config:
wandb: 	batch_size: 256
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep


train_loss,2.2938
val_loss,2.26363


train_loss,▆▄▆▆▅▅▄▅▆█▄▅▄▅▅▅▃▃▅▇▄▄▁▅▃▆▂▂▄▄▄▂▃▃▆▂▃▃▄▂
val_loss,█▆▄▃▁


wandb: Agent Starting Run: uua6i2cc with config:
wandb: 	batch_size: 256
wandb: 	epochs: 5
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep
